In [1]:
# Imports as always...
import numpy as np
from sqif import CVP, solve_cvp, integer_matrix_to_numpy, primes

# Ignore warnings.
import warnings
warnings.filterwarnings('ignore')

First, we need some lattice vectors obtained as solutions to the CVP.

In [35]:
cvp = CVP()

N = 1961
cvp.generate_cvp(N=N, l=1, c=1.5, seed=42)

lattice_vectors, outcome_probabilities, b_op = solve_cvp(
    cvp, n_samples=10000, delta=.75, p=1, min_method='Nelder-Mead', verbose=False
)

b_op_dist_to_t = np.linalg.norm(b_op - cvp.t)
for out, prob in zip(lattice_vectors, outcome_probabilities):
    extra = ''
    dist_to_t = np.linalg.norm(out - cvp.t)
    if np.all(out == b_op):
        extra = '[Approx. solution]'
    elif dist_to_t < b_op_dist_to_t:
        extra = '[Better]'
        
    print(f'{out} with p={prob:.3f}; dist. to t: {dist_to_t:.3f} {extra}')

[  3   5   0 241] with p=0.328; dist. to t: 5.916 [Better]
[  0   4   4 242] with p=0.277; dist. to t: 6.000 [Approx. solution]
[  3   2   4 238] with p=0.260; dist. to t: 5.745 [Better]
[ -1   6   2 239] with p=0.097; dist. to t: 6.481 
[  4   3   2 244] with p=0.025; dist. to t: 6.708 
[  7   1   2 240] with p=0.005; dist. to t: 7.348 
[  6   3   0 237] with p=0.004; dist. to t: 7.348 
[  2   4   2 235] with p=0.004; dist. to t: 7.000 


Then we reduce these lattice vectors to $u,v$ pairs.

In [41]:
def lattice_vectors_to_u_v_pairs(lattice_vectors, B):
    """
    Reduce a set of lattice vectors to a set of u, v pairs.

    :param lattice_vectors: numpy.ndarray of lattice vectors.
    :param B: (numpy.ndarray) The basis matrix of the prime lattice. 
    """

    # Compute the pseudo-inverse of the prime lattice's basis matrix.
    B_pinv = np.linalg.pinv(np.asarray(B, dtype=int))

    # Left-multiply the lattice vectors by this pseudo-inverse to yield the prime expnents.
    # Round these to the nearest integer (integer pairs only!).
    prime_exponents = np.rint(lattice_vectors @ B_pinv.T).astype(int)
    print(f'Prime exponents:\n{prime_exponents}')

    # Find the smooth pairs corresponding to these exponents.
    first_n_primes = primes[:prime_exponents.shape[1]]
    u_exp = np.multiply(prime_exponents, (prime_exponents > 0).astype(int))
    v_exp = np.multiply(-prime_exponents, (prime_exponents < 0).astype(int))
    u = np.prod(np.power(first_n_primes, u_exp), axis=1)
    v = np.prod(np.power(first_n_primes, v_exp), axis=1)

    return np.stack((u, v), axis=-1)

u_v_pairs = lattice_vectors_to_u_v_pairs(lattice_vectors, integer_matrix_to_numpy(cvp.B))
u_v_pairs

Prime exponents:
[[ 3  5  0]
 [ 0  4  2]
 [ 3  2  2]
 [-1  6  1]
 [ 4  3  1]
 [ 7  1  1]
 [ 6  3  0]
 [ 2  4  1]]


array([[1944,    1],
       [2025,    1],
       [1800,    1],
       [3645,    2],
       [2160,    1],
       [1920,    1],
       [1728,    1],
       [1620,    1]])

Then we reduce these $u,v$ pairs to smooth-relation pairs.

In [37]:
# Helper function.
def is_smooth(x, smooth_bound):
    first_n_primes = primes[:smooth_bound]

    for p in first_n_primes:
        while x % p == 0:
            x //= p

    return x == 1

def u_v_pairs_to_sr_pairs(u_v_pairs, N, smooth_bound):
    """
    Reduce a set of u, v pairs into smooth-relation pairs below a given smooth bound.
    
    :param u_v_pairs: numpy.ndarray of u, v pairs.
    :param N: Semi-prime to be factored.
    :param smooth_bound: The smooth bound (for alpha = 1, the lattice dimension).
    """
    
    # Give back the u, v pairs that form smooth-relation pairs under the given smooth bound.
    return [
        tuple(u_v) 
        for u_v in u_v_pairs
        if is_smooth(
            # | u - N*v | must be p_n-smooth, for given smooth bound p_n.
            abs(int(u_v[0]) - N * int(u_v[1])), smooth_bound
        )
    ]

# See equation (S11) in Yan et al. (2022) for the definition of the smoothness bound.
# In particular, we take alpha = 1, as they suggest.
smooth_bound = (np.log2(N) / np.log2(np.log2(N))).astype(int)
# This gives the lattice dimension.

u_v_pairs_to_sr_pairs(u_v_pairs, N, smooth_bound)

[(2025, 1)]

These found smooth-relation pairs are then involved in a system of linear equations, so we require at least ? sr-pairs.